In [31]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [32]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-r1-distill-qwen-32b", model_provider="groq")

In [33]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [34]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [37]:
import json
from pathlib import Path
from pprint import pprintz

file_path = 'resume.json'
data = json.loads(Path(file_path).read_text())
print(data)

{'personal_info': {'name': 'Xiaojian Zhuang', 'github': 'github@pandalow', 'email': 'zxj000hugh@gmail.com', 'mobile': '+353-0874796931'}, 'summary': 'Driven by a passion for creating impactful products, I transitioned from product management, where I spent 6 years refining user-focused solutions, to software engineering. This journey led me to Ireland to gain the technical expertise needed to bring ideas to life. My background spans experiences from front-end to back-end and holistic database system, all shaped by agile environments and fast-paced, high-growth companies. Collaboration and adaptability define my work style; I thrive in dynamic teams, balancing diverse needs while delivering value. My enthusiasm for learning and experimenting fuels personal projects that explore new possibilities, allowing me to innovate and grow continuously.', 'strength_skills': ['Java', 'Python', 'Spring', 'React', 'Interaction Design', 'Structural abstraction', 'Empathy', 'Data analysis', 'Adaptabili

In [35]:
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from typing_extensions import TypedDict, List
from langchain.schema import Document
from langgraph.graph import START, StateGraph

loader = JSONLoader(file_path="resume.json", jq_schema='.content')

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [36]:
response = graph.invoke({"question": "How many years of experience do you have?"})
print(response["answer"])

<think>
Alright, the user is asking about my years of experience. Looking at the context provided, it's a resume-like document listing skills, roles, and responsibilities. 

I see two main roles: Graduate Teaching Assistant starting in September 2024 and Technical Product Manager from May 2021 to June 2023. 

Since the assistant role is current, I can't calculate the exact duration yet. The product manager role lasted about two years. 

Combining both roles, the total experience is around two years and ongoing.
</think>

Based on the context provided, I don't have information about the exact number of years of experience. The context lists roles such as Graduate Teaching Assistant (Sep 2024 - Present) and Technical Product Manager (May 2021 - Jun 2023), but it doesn't specify the total years of experience.
